In [3]:
# coding = 'utf-8'
import numpy as np
import pandas as pd


def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result


def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result


def main():
    y = np.random.randint(2, size=(5000, 1))
    x = np.random.randint(10, size=(5000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
    result_1 = target_mean_v1(data, 'y', 'x')
    result_2 = target_mean_v2(data, 'y', 'x')

    diff = np.linalg.norm(result_1 - result_2)
    print(diff)


if __name__ == '__main__':
    main()

0.0


In [5]:
%load_ext cython


In [6]:
%%cython

import numpy as np
cimport numpy as cnp
cimport cython
from cython.parallel import prange

# 最好每个都加上类型
# 报错: Indexing Python object not allowed without gil

# 不添加装饰器会出现警告 Use boundscheck(False) for faster access
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cnp.ndarray[double] target_mean_v7(cnp.ndarray[long,ndim=2] data):
    # get row numbers
    cdef int n = data.shape[0]
    # result to return 
    cdef cnp.ndarray[double] result = np.zeros(n,dtype=np.double)
    # get y and x 
    cdef cnp.ndarray[long] y = data[:,0]
    cdef cnp.ndarray[long] x = data[:,1]
    # define map
    cdef cnp.ndarray[long] value_map = np.zeros(n).astype(np.int_)
    cdef cnp.ndarray[long] count_map = np.zeros(n).astype(np.int_)
    
    cdef int i
    for i in prange(n, nogil=True):
        value_map[x[i]] += y[i]
        count_map[x[i]] += 1
    for i in prange(n, nogil=True):
        result[i] = (value_map[x[i]] - y[i]) / (count_map[x[i]] - 1)
    return result

In [11]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [12]:
data2 = data.values.astype(np.int_)

In [13]:
%%timeit
target_mean_v7(data2)

47.8 µs ± 753 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
